# Group Case Study : Loan Default Prediction 
#### Group Facilitator Name : Prabhat Mishra ( prabhatm0307@gmail.com)
#### Team Member: Deepanshu Pradhan (pradhand1@gmail.com)

## Exploratory Data Analysis (EDA) to understand Loan Default

#### Business Understanding

A consumer finance company specialised in lending various types of loans to urban customers. It is the largest online loan marketplace, facilitating personal loans, business loans, and financing of medical procedures. Borrowers can easily access lower interest rate loans through a fast online interface. 

When a loan application is received by the company, the company must make a decision to approve the loan based on the profile of the applicant. Two kinds of risks are linked to the choice of the bank

1. If the applicant is inclined to repay the loan, then not approving the loan will result in a business loss to the company
2. If the applicant is unlikely to repay the loan, i.e. if the applicant is inclined to default, then approving the loan may result in a financial loss to the company.

If the firm approves the loan, there are three possible scenarios outlined below:

   i. <b>Fully paid</b>: Applicant has fully paid the loan (the principal and the interest rate)
   
   ii.<b>Current</b>: Applicant is in the process of paying the instalments, i.e. the tenure of the loan is not yet completed.
   
   iii.<b>Charged-off</b>: Applicant has not paid the instalments in due time for a long period of time, i.e. he/she has defaulted on the loan 

#### Objective

Lending loans to ‘risky’ applicants is the largest source of financial loss (called credit loss). In other words, borrowers who default cause the largest amount of loss to the lenders.Such customers labelled as <b> 'charged-off' </b> are the <b>'defaulters'</b>. 

The objective is to understand the driving factors (or driver variables) behind loan default. The company can utilise this knowledge for its portfolio and risk assessment. 


# Exploratory Data Analysis (EDA) into below sections

#### 1. Data Sourcing

    i.  Loading Data into dataframe    
    ii. Defining Generic Functions for plotting and metadata
    
    
#### 2. Data Cleaning


    A. Null Value Analysis
        i.  Identify and drop columns with completly missing data
        ii. Identify and drop columns with more than 75% percentage missing data
        iii.Identify and drop columns having single value as they will not add any value to the analysis
        iv. Analyze Columns and identify/drop unnecessary columns
        
    B. Data Type Check for columns
    
    C. Converting object columns into correct datatype
        i. Converting the text columns from object to string
        ii.Converting the columns from % column to float post striping 
   
    D. Drop Duplicates Records
    
    E. Impute Null Values
    
    F. Derived Columns
        i.  Derive Month and Year from Issue Date
        ii. Derive Loan amount and Annual Income ratio
        iii.Adding frequency column for plotting 

#### 3. Univariate Analysis

    A. Continous Variables
        i.  Correlation
        ii. Heat Map
        iii.Perform univariate analysis on one of the strongly correlated columns
        
    B. Categorical Variables
    
    
#### 4. Bivariate/Multivariate Analysis

## 1. Data Sourcing
### i. Loading Data into dataframe

In [1]:
import pandas as pd
import numpy as np
import cufflinks as cf
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from scipy import special 
import matplotlib.pyplot as plt
import seaborn as sns
import math
import sys
import cchardet as chardet
from pandas.api.types import is_string_dtype, is_numeric_dtype
import lux

sns.set(style="whitegrid")
pd.set_option('display.max_columns', 100)
py.offline.init_notebook_mode(connected=True) # plotting in offilne mode 
cf.set_config_file(offline=False, world_readable=True, theme='ggplot')
pd.set_option('display.max_colwidth', 1) # make sure data and columns are displayed correctly withput purge
pd.options.display.float_format = '{:20,.2f}'.format # display float value with correct precision 
with open("loan.csv", 'rb') as rawdata:
    result = chardet.detect(rawdata.read(10000))
encoding=result['encoding']
print(encoding)

ASCII


In [2]:
encoding="UTF-8"
loan_df = pd.read_csv("loan.csv",low_memory = False,encoding = encoding)
loan_df.shape

(39717, 111)

### ii. Defining Generic Functions for plotting and metadata

In [3]:
def plot_bar_chart(plotting_frame,x_column,y_column) :
            
        x_axis_title = x_column.title()
        y_axis_title = y_column.title()
        
        graph_title = "Bar Chart [" + x_axis_title.title() + " Vs " + y_axis_title.title() + "]"
        
        layout = go.Layout(
             title = graph_title,
             yaxis=dict(
                title=y_axis_title
             ),
             xaxis=dict(
                 title=x_axis_title
             )
        )

        data_to_be_plotted = [
            go.Bar(
                x=plotting_frame[x_column], 
                y=plotting_frame[y_column]
            )
        ]


        figure = go.Figure(data=data_to_be_plotted,layout=layout)
        py.offline.iplot(figure)
        
        
def plot_pie_chart(plotting_frame,x_column,y_column) : 
        
        labels = plotting_frame[x_column].tolist()
        values = plotting_frame[y_column].tolist()

        trace = go.Pie(labels=labels, values=values)

        py.offline.iplot([trace])

        
def plot_box_chart(dataframe) :
    data = []
    for index, column_name in enumerate(dataframe) :
        data.append(
        go.Box(
            y=dataframe.iloc[:, index],
            name=column_name
         ))   
        
    layout = go.Layout(
    yaxis=dict(
        title="Frequency",
        zeroline=False
    ),
       boxmode='group'
    )
    
    fig = go.Figure(data=data, layout=layout)    
    py.offline.iplot(fig) 
    
def plot_group_bar_chart(plot,col,hue) : 
    hue_col = pd.Series(data = hue)
    fig, ax = plt.subplots()
    width = len(plot[col].unique()) + 6 + 5*len(hue_col.unique())
    fig.set_size_inches(width , 10)
    ax = sns.countplot(data = loan_plot, x= col, order=plot[col].value_counts().index,hue = hue,palette="Set2") 
    
    for p in ax.patches:
                # Some segment wise value we are getting as Nan as respective value not present to tackle the Nan using temp_height
                temp_height = p.get_height()
                
                if math.isnan(temp_height):
                    temp_height = 0.01
                    
                
                ax.annotate('{:1.1f}%'.format((temp_height*100)/float(len(loan_plot))), (p.get_x()+0.05, temp_height+20)) 
    
    plt.show()
    
# Method to determine metadata of any dataframe
def getMetadata(df) :
    metadata_info = pd.DataFrame({
        'Total_Element': df.count(), # total elements in columns            
        'Datatype' : df.dtypes, # data types of columns
        'Null_Count': df.isnull().sum(), # total null values in columns
        'Null_Percentage': df.isnull().sum()/len(df) * 100 # percentage of null values
                       })
    return metadata_info

def yearCal(date) :
    temp = date.split('-')[1]
    lenght = len(temp)
    if lenght == 2 :
        temp = "20"+temp
    else :
        temp = "200"+temp
        
    return temp

## 2. Data Cleaning
### A. Null Values Analysis

In [4]:
loan_metadata = getMetadata(loan_df)
loan_metadata_group = loan_metadata.groupby("Null_Percentage").count().reset_index()
loan_metadata_group.sort_values(["Null_Percentage"], axis=0,ascending=False, inplace=True)
plot_pie_chart(loan_metadata_group,"Null_Percentage","Null_Count")

<font color='green'/>**Analysis** - 54 columns are completely null and can be removed from further analysis

### i. Identify and drop columns with completly missing data

In [5]:
completly_missing_data = loan_metadata[loan_metadata["Null_Percentage"] == 100.0]
drop_missing_column = completly_missing_data.index.tolist()
print("Null Columns before deleting  : " + str(loan_df.shape[1]))
loan_df.drop(drop_missing_column,inplace=True,axis=1)
print("Null Columns after deleting : " + str(loan_df.shape[1]))

Null Columns before deleting  : 111
Null Columns after deleting : 57


<font color='green'/>**Analysis** - Approx 50% are dropped from further analysis

### ii. Identify and drop columns with more than 75% percentage missing data

In [6]:
missing_data_greater_75 = loan_metadata[(loan_metadata["Null_Percentage"] > 75.0) & (loan_metadata["Null_Percentage"] < 100.0)]
drop_missing_column_75 = missing_data_greater_75.index.tolist()
loan_df.drop(drop_missing_column_75,inplace=True,axis=1)
print("Shape after deleting 75% columns ",loan_df.shape ,"rows & columns.")

Shape after deleting 75% columns  (39717, 55) rows & columns.


### iii. Identify and drop columns having single value as they will not add any value to our analysis

In [7]:
unique_value = loan_df.nunique()
col_with_only_one_value = unique_value[unique_value.values == 1]
col_to_drop = col_with_only_one_value.index.tolist()
loan_df.drop(col_to_drop, axis =1, inplace=True)
print("Shape after deleting unique value columns ",loan_df.shape ,"rows & columns.")

Shape after deleting unique value columns  (39717, 46) rows & columns.


### iv. Analyize Columns and identify/drop unnecessary columns

In [8]:
loan_df.head(2)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:401: UserWarning:
Lux detects that the attribute 'earliest_cr_line' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
If earliest_cr_line is not a temporal attribute, please use override Lux's automatically detected type:
	df.set_data_type({'earliest_cr_line':'quantitative'

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,url,desc,purpose,title,zip_code,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,1077501,1296599,5000,5000,"4,975.00",36 months,10.65%,162.87,B,B2,NaN,10+ years,RENT,"24,000.00",Verified,Dec-11,Fully Paid,https://lendingclub.com/browse/loanDetail.action?loan_id=1077501,Borrower added on 12/22/11 > I need to upgrade my business technologies.<br>,credit_card,Computer,860xx,AZ,27.65,0,Jan-85,1,NaN,3,0,13648,83.70%,9,0.00,0.00,"5,863.16","5,833.84","5,000.00",863.16,0.00,0.00,0.00,Jan-15,171.62,May-16,0.00
1,1077430,1314167,2500,2500,"2,500.00",60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,"30,000.00",Source Verified,Dec-11,Charged Off,https://lendingclub.com/browse/loanDetail.action?loan_id=1077430,Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike. I only need this money because the deal im looking at is to good to pass up.<br><br> Borrower added on 12/22/11 > I plan to use this money to finance the motorcycle i am looking at. I plan to have it paid off as soon as possible/when i sell my old bike.I only need this money because the deal im looking at is to good to pass up. I have finished college with an associates degree in business and its takingmeplaces<br>,car,bike,309xx,GA,1.00,0,Apr-99,5,NaN,3,0,1687,9.40%,4,0.00,0.00,"1,008.71","1,008.71",456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.00


<font color='green'/>**Analysis** - By looking at the data , below fields doesn't add any value to our case study - 
1. url  : URL for the LC page with listing data.
2. desc : Loan description provided by the borrower
3. zip_code : The first 3 numbers of the zip code provided by the borrower in the loan application.
4. id : A unique LC assigned ID for the loan listing.
5. member_id :  This is a unique id and has no use

In [9]:
col_to_drop = ["url","desc","zip_code","id","member_id"]
loan_df.drop(col_to_drop,inplace=True,axis=1)
loan_df.head(5)
print("Shape after deleting unnecessary columns ",loan_df.shape ,"rows & columns.")

Shape after deleting unnecessary columns  (39717, 41) rows & columns.


### B. Data Type Check for columns

In [10]:
loan_data_type = getMetadata(loan_df)
loan_data_type["Datatype"].value_counts()

object     17
float64    16
int64      8 
dtype: int64


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/lib/pretty.py:700: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


In [11]:
loan_data_type_float = loan_data_type[loan_data_type["Datatype"] == "float64"]
loan_data_type_int = loan_data_type[loan_data_type["Datatype"] == "int64"]
loan_data_type_object = loan_data_type[loan_data_type["Datatype"] == "object"]

In [12]:
### float columns
loan_data_type_float_group = loan_data_type_float.groupby("Null_Percentage").count().reset_index()
plot_pie_chart(loan_data_type_float_group,"Null_Percentage","Null_Count")
loan_data_type_float

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

<font color='green'/>**Analysis** - 87% of float datatypes have correct values and can be used for analyis 

In [13]:
## int columns
loan_data_type_int_group = loan_data_type_int.groupby("Null_Percentage").count().reset_index()
plot_pie_chart(loan_data_type_int_group,"Null_Percentage","Null_Count")
loan_data_type_int

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,Total_Element,Datatype,Null_Count,Null_Percentage
loan_amnt,39717,int64,0,0.00
funded_amnt,39717,int64,0,0.00
delinq_2yrs,39717,int64,0,0.00
inq_last_6mths,39717,int64,0,0.00
open_acc,39717,int64,0,0.00
pub_rec,39717,int64,0,0.00
revol_bal,39717,int64,0,0.00
total_acc,39717,int64,0,0.00


<font color='green'/>**Analysis** - No missing values for integer columns

In [14]:
## object columns
loan_data_type_object_group = loan_data_type_object.groupby("Null_Percentage").count().reset_index()
plot_pie_chart(loan_data_type_object_group,"Null_Percentage","Null_Count")
loan_data_type_object

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

<font color='green'/>**Analysis** - ~65% of float datatypes have correct values and can be used for directly for analyis  post correction of datatype. Rest columns need to be imputed and converted

In [15]:
### for object dtype identify null and not null columns 
columns_to_convert_not_null = loan_data_type_object[loan_data_type_object["Null_Count"] == 0].index.tolist()
columns_to_convert_null = loan_data_type_object[loan_data_type_object["Null_Count"] > 0].index.tolist()
print(columns_to_convert_not_null)
print(columns_to_convert_null)

['term', 'int_rate', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'addr_state', 'earliest_cr_line']
['emp_title', 'emp_length', 'title', 'revol_util', 'last_pymnt_d', 'last_credit_pull_d']


### C. Converting object columns into correct datatype

In [16]:
loan_df[columns_to_convert_not_null].describe(include="all")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:401: UserWarning:
Lux detects that the attribute 'earliest_cr_line' may be temporal.
To display visualizations for these attributes accurately, please convert temporal attributes to Pandas Datetime objects using the pd.to_datetime function and provide a 'format' parameter to specify the datetime format of the attribute.
For example, you can convert a year-only attribute (e.g., 1998, 1971, 1982) to Datetime type by specifying the `format` as '%Y'.

Here is a starter template that you can use for converting the temporal fields:
	df['earliest_cr_line'] = pd.to_datetime(df['earliest_cr_line'], format='<replace-with-datetime-format>')

See more at: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html
If earliest_cr_line is not a temporal attribute, please use override Lux's automatically detected type:
	df.set_data_type({'earliest_cr_line':'quantitative'

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

In [17]:
loan_df[columns_to_convert_null].describe(include="all")

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:250: UserWarning:
Lux detects that the attribute 'last_credit_pull_d' maybe contain mixed type.
To visualize this attribute, you may want to convert the 'last_credit_pull_d' into a uniform type as follows:
	df['last_credit_pull_d'] = df['last_credit_pull_d'].astype(str)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:250: UserWarning:
Lux detects that the attribute 'revol_util' maybe contain mixed type.
To visualize this attribute, you may want to convert the 'revol_util' into a uniform type as follows:
	df['revol_util'] = df['revol_util'].astype(str)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/executor/PandasExecutor.py:250: UserWarning:
Lux detects that the attribute 'emp_length' maybe contain mixed type.
To visualize this attribute, you may want to convert the 'emp_length' into 

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

#### i. Converting the text columns from object to string

In [18]:
loan_df[['last_pymnt_d','last_credit_pull_d','term','grade', 
         'sub_grade', 'home_ownership','verification_status', 
         'loan_status', 'purpose', 'addr_state',
         'emp_title', 'emp_length', 'title','issue_d','earliest_cr_line']]=loan_df[['last_pymnt_d','last_credit_pull_d','term','grade', 
                                                       'sub_grade', 'home_ownership','verification_status', 
                                                       'loan_status', 'purpose', 'addr_state','emp_title', 
                                                       'emp_length', 'title','issue_d','earliest_cr_line']].astype("string")

In [19]:
loan_df.head(2)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,5000,5000,"4,975.00",36 months,10.65%,162.87,B,B2,<NA>,10+ years,RENT,"24,000.00",Verified,Dec-11,Fully Paid,credit_card,Computer,AZ,27.65,0,Jan-85,1,NaN,3,0,13648,83.70%,9,0.00,0.00,"5,863.16","5,833.84","5,000.00",863.16,0.00,0.00,0.00,Jan-15,171.62,May-16,0.00
1,2500,2500,"2,500.00",60 months,15.27%,59.83,C,C4,Ryder,< 1 year,RENT,"30,000.00",Source Verified,Dec-11,Charged Off,car,bike,GA,1.00,0,Apr-99,5,NaN,3,0,1687,9.40%,4,0.00,0.00,"1,008.71","1,008.71",456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.00


#### ii. Converting the text columns from % column to float post striping 

In [20]:
loan_df["int_rate"] = loan_df["int_rate"].apply(lambda x: x.rstrip("%")).astype(float)
loan_df["revol_util"]=loan_df["revol_util"].astype(str).apply(lambda x: x.replace("%","")).astype(float)
loan_df.head(2)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,emp_title,emp_length,home_ownership,annual_inc,verification_status,issue_d,loan_status,purpose,title,addr_state,dti,delinq_2yrs,earliest_cr_line,inq_last_6mths,mths_since_last_delinq,open_acc,pub_rec,revol_bal,revol_util,total_acc,out_prncp,out_prncp_inv,total_pymnt,total_pymnt_inv,total_rec_prncp,total_rec_int,total_rec_late_fee,recoveries,collection_recovery_fee,last_pymnt_d,last_pymnt_amnt,last_credit_pull_d,pub_rec_bankruptcies
0,5000,5000,"4,975.00",36 months,10.65,162.87,B,B2,<NA>,10+ years,RENT,"24,000.00",Verified,Dec-11,Fully Paid,credit_card,Computer,AZ,27.65,0,Jan-85,1,NaN,3,0,13648,83.70,9,0.00,0.00,"5,863.16","5,833.84","5,000.00",863.16,0.00,0.00,0.00,Jan-15,171.62,May-16,0.00
1,2500,2500,"2,500.00",60 months,15.27,59.83,C,C4,Ryder,< 1 year,RENT,"30,000.00",Source Verified,Dec-11,Charged Off,car,bike,GA,1.00,0,Apr-99,5,NaN,3,0,1687,9.40,4,0.00,0.00,"1,008.71","1,008.71",456.46,435.17,0.00,117.08,1.11,Apr-13,119.66,Sep-13,0.00


In [21]:
print(loan_df.shape)
loan_data_type = getMetadata(loan_df)
loan_data_type["Datatype"].value_counts()

(39717, 41)
float64    18
string     15
int64      8 
dtype: int64


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/lib/pretty.py:700: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


### D. Drop Duplicates Records

In [22]:
print("Records before dropping duplicates  : " + str(loan_df.shape[0]))
loan_df.drop_duplicates(keep=False,inplace=True)
print("Records after dropping duplicates  : " + str(loan_df.shape[0]))

Records before dropping duplicates  : 39717
Records after dropping duplicates  : 39717


### D. Imputing Null Values

In [23]:
impute_columns = getMetadata(loan_df)
impute_columns = impute_columns[impute_columns["Null_Count"] > 0]
impute_columns.head(20)

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Unexpected error in rendering Lux widget and recommendations. Falling back to Pandas display.
Please report the following issue on Github: https://github.com/lux-org/lux/issues 

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/core/frame.py:700: UserWarning:Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pandas/core/algorithms.py", line 2123, in safe_sort
    sorter = values.argsort()
TypeError: Cannot interpret 'StringDtype' as a data type

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/lux/core/frame.py", line 651, in _repr_html_
    self.maintain_recs()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/py

,Total_Element,Datatype,Null_Count,Null_Percentage
emp_title,37258,string,2459,6.19
emp_length,38642,string,1075,2.71
title,39706,string,11,0.03
mths_since_last_delinq,14035,float64,25682,64.66
revol_util,39667,float64,50,0.13
last_pymnt_d,39646,string,71,0.18
last_credit_pull_d,39715,string,2,0.01
pub_rec_bankruptcies,39020,float64,697,1.75


<font color='green'/>**Analysis** -By looking at above we need to handle **emp_length** as this will be out main feature for analysis and we can delete **title and emp_title** as they are non relevant

In [24]:
## Dropping title and emp title 
to_drop = ["title","emp_title"]
loan_df.drop(to_drop , inplace=True,axis=1)

In [25]:
loan_df_temp = loan_df.filter(impute_columns.index.tolist())
loan_df_temp.head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,emp_length,mths_since_last_delinq,revol_util,last_pymnt_d,last_credit_pull_d,pub_rec_bankruptcies
0,10+ years,NaN,83.70,Jan-15,May-16,0.00
1,< 1 year,NaN,9.40,Apr-13,Sep-13,0.00
2,10+ years,NaN,98.50,Jun-14,May-16,0.00
3,10+ years,35.00,21.00,Jan-15,Apr-16,0.00
4,1 year,38.00,53.90,May-16,May-16,0.00


<font color='green'/>**Analysis** - For missing employment length, imputing it with Unknown value

In [26]:
## Imputing employee length with unknown value
## Imputing continous values with median
loan_df["emp_length"].fillna('Unknown',inplace=True)
loan_df["mths_since_last_delinq"].fillna((loan_df["mths_since_last_delinq"].median()),inplace=True)
loan_df["revol_util"].fillna((loan_df["revol_util"].median()),inplace=True)
loan_df["pub_rec_bankruptcies"].fillna((loan_df["pub_rec_bankruptcies"].median()),inplace=True)

### E. Derived Metrics

#### i. Derive Month and Year from Issue Date

In [27]:
loan_df['issue_month']  = loan_df['issue_d'].apply(lambda date:date.split('-')[0]).astype("string")
loan_df['issue_year'] = loan_df['issue_d'].apply(yearCal).astype("string")
loan_df[['issue_d','issue_month','issue_year']].head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,issue_d,issue_month,issue_year
0,Dec-11,Dec,2011
1,Dec-11,Dec,2011
2,Dec-11,Dec,2011
3,Dec-11,Dec,2011
4,Dec-11,Dec,2011


#### ii. Derive Loan amount and Annual Income ratio

In [28]:
loan_df['loan_amt_income_ratio']=loan_df['loan_amnt']/loan_df['annual_inc']
loan_df[['loan_amt_income_ratio','loan_amnt','annual_inc']].head()

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/IPython/core/formatters.py:345: UserWarning:
Lux defaults to Pandas when there are no valid actions defined.


,loan_amt_income_ratio,loan_amnt,annual_inc
0,0.21,5000,"24,000.00"
1,0.08,2500,"30,000.00"
2,0.20,2400,"12,252.00"
3,0.20,10000,"49,200.00"
4,0.04,3000,"80,000.00"


#### iii. Adding frequency column for plotting

In [29]:
loan_df["frequency"] = loan_df["loan_amnt"] - loan_df["loan_amnt"]

## 3.Univariate Analysis

In [30]:
df=loan_df
num_list = []
cat_list = []

for column in df:
    if is_numeric_dtype(df[column]):
        num_list.append(column)
    elif is_string_dtype(df[column]):
        cat_list.append(column)    

print("Category Columns:",cat_list)
print("Continous Columns:",num_list)

Category Columns: ['term', 'grade', 'sub_grade', 'emp_length', 'home_ownership', 'verification_status', 'issue_d', 'loan_status', 'purpose', 'addr_state', 'earliest_cr_line', 'last_pymnt_d', 'last_credit_pull_d', 'issue_month', 'issue_year']
Continous Columns: ['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'int_rate', 'installment', 'annual_inc', 'dti', 'delinq_2yrs', 'inq_last_6mths', 'mths_since_last_delinq', 'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_amnt', 'pub_rec_bankruptcies', 'loan_amt_income_ratio', 'frequency']


In [32]:
loan_df

Button(description='Toggle Pandas/Lux', layout=Layout(top='5px', width='140px'), style=ButtonStyle())

Output()

### 1. Continous Variables
#### i. Correlation/Heatmap

### 2. Categorical Variables

## 4. Bivariate/Multivariate Analysis 
### i. Plot b/w driving factors
### ii. Plot against target variable

## 5. Conclusion